In [23]:
# Entrer l'acteur sur lequel on veut lancer l'analyse globale
acteur = 'pinel'

In [24]:
import shutil, re, pandas
from pandas import *
import os
from os import listdir, chdir, path
from pathlib import Path

def nlp(file, souscorpus=False):
    print("Corpus traité : "+ file)
    #Lire le corpus
    print("Lire le corpus")

    if souscorpus:
        base_path = path.join('../03-corpus/2-sous-corpus/', acteur)
        file_path = path.join(base_path, acteur + '_' + file +  '.csv')

    else:
        base_path = '../03-corpus/2-data/1-fr/'
        file_path = path.join(base_path, acteur + '.csv')
   
    with open(file_path, "r", encoding = "UTF-8") as f:
            data = read_csv(file_path)
            text = data['text'].tolist()
            corpus = " ".join([(re.sub('\d', '', t.strip('\n').lower().replace('’', '\''))) for t in text])

    nb_docs = len(text)

    print("On a un corpus de {} documents.".format(nb_docs))

    # Prétraitement
    print("Prétraitement")
    import nltk
    #nltk.download(['popular'])

    punct = '!#$%&()*+,-/:;<=>?@[\]^_{|}~©'

    for t in punct:
        corpus = corpus.replace(t, ' ').replace("  ", " ")


    # Filtrage - MWE Stopwords
    file_path = '../04-filtrage/mwe_stopwords.txt'

    with open (file_path, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]

    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, '')

    # Tokeniser
    print("Tokenisation")
    from nltk.tokenize import RegexpTokenizer

    # Seulement les caractères alphabétiques
    tokenizer_re = RegexpTokenizer(r"\w\'|\w+")

    tokens = tokenizer_re.tokenize(corpus)
    len_corpus = len(tokens)

    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len_corpus))

    corpus = " ".join(tokens).replace("' ", "'") # Pour "forcer" le TreeTagger à tokeniser comme on veut

    # POS Tagging (TreeTagger)
    print("Étiquetage morphosyntaxique")

    import treetaggerwrapper
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

    # Mapper les étiquettes du TreeTagger à celles du Lefff
    file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

    with open(file_path) as f:
        csv = read_csv(f)

    treeTag = [term for term in csv['TreeTagger'].tolist()] 
    lefff = [term for term in csv['Lefff'].tolist()]

    mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

    tagged = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]
    print("(Validation - On a {} tokens taggés).".format(len(tagged)))

    # Lemmatisation
    print("Lemmatisation")
    from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
    lemmatizer = FrenchLefffLemmatizer()

    lemmas = []
    for term in tagged:
        term_l = []
        if lemmatizer.lemmatize(term[0], term[1]) == []:
            term_l = [lemmatizer.lemmatize(term[0]), term[1]]
        
        elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
            term_l  = [lemmatizer.lemmatize(term[0], term[1]), term[1]]

        else:
            term_l = list(lemmatizer.lemmatize(term[0], term[1])[0])
        
        lemmas.append(term_l)
    
    print("(Validation - On a {} tokens lemmatisés).".format(len(lemmas)))

    base_path = '../04-filtrage/output/'
    file_path = path.join(base_path, acteur)
    if souscorpus:
        file_path = path.join(file_path, file)

    # Extraction de collocations (n-grammes)
    from nltk.util import ngrams
    from nltk.util import bigrams
    from nltk.util import everygrams
    from nltk.probability import FreqDist
    ngrammes = list(everygrams(tagged, min_len=2, max_len=5))
    ngrammes_lemmatized = list(everygrams(lemmas, min_len=2, max_len=5))
        
    # Extraction des patrons syntaxiques des ngrammes
    def extract_patterns(ngrammes):
        patterns = []

        for ng in ngrammes:
            phrase = []
            pattern = []
            for t in ng:
                phrase.append(t[0]) # token
                pattern.append(t[1]) # POS tag

            patterns.append([phrase, pattern])
            
        return patterns

    phrases = extract_patterns(ngrammes)
    phrases_lemmatized = extract_patterns(ngrammes_lemmatized)

    # Extraction des fréquences d'occurrence des ngrammes
    freq = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])
    freq_lemmatized = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases_lemmatized])

    # Filtrage 
    # Stopwords lemmatisés
    file_path = '../04-filtrage/stopwords_lemmatized.txt'
    with open(file_path, 'r', encoding="utf-8") as f:
        stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

    # Stopwords fréquents en français (non lemmatisés)
    file_path = "../04-filtrage/stopwords.txt"
    with open(file_path, 'r', encoding="utf-8") as f:
        stopwords = [t.lower().strip('\n') for t in f.readlines()]


    # Stopwords fréquents en anglais (non lemmatisés)
    file_path = '../04-filtrage/stop_words_english.txt'
    with open(file_path, 'r', encoding="utf-8") as f:
        stopwords += [t.lower().strip('\n') for t in f.readlines()]

    print("Filtrage - n-grammes")
    def filtrer_phrases(phrases, freq):
        output = []
        for term in phrases:
                exp = " ".join(term[0]).replace("' ", "'")
                f = freq[exp]
                
                # f > 10 and - sans filtrer par fréquence 
                if  f > 10 and not term[0][0] in stopwords and len(term[0][0]) > 2 \
                and not term[0][-1] in stopwords and len(term[0][-1]) > 2 : 
                        pattern = " ".join(term[1])            
                        output.append([exp, pattern, f])  
                         
        return output

    phrases = filtrer_phrases(phrases, freq)
    phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

    print("Après le filtrage, on a {} occurrences de nos n-grammes.".format(len(phrases))) 

    # 1e extrant : n-grammes filtrés
    # On prépare notre dossier pour stocker les output des prochaines étapes
    base_path = path.join('../04-filtrage/output/', acteur)
    if souscorpus:
        base_path = path.join(base_path, file)
    file_path = path.join(base_path, file)

    Path(base_path).mkdir(parents=True, exist_ok=True) 

    def tabCSV(liste, titre):
        tab = DataFrame(liste, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates()
        tab.sort_values(["Fréquence"], 
                            axis=0,
                            ascending=[False], 
                            inplace=True)


        tab.to_csv(file_path + titre)
        return tab.values.tolist()

    phrases = tabCSV(phrases, '_n-grams.csv')
    phrases_lemmatized = tabCSV(phrases_lemmatized, '_n-grams-lemmatized.csv')

    print("Après filtrage, on a {} n-grammes uniques.".format(len(phrases)))

    # Filtrage - Patrons syntaxiques
    print("Filtrage - n-grammes - par patrons syntaxiques")
    file_mesh = '../04-filtrage/mesh_patterns-fr.csv'

    with open (file_mesh, 'r') as f:
        patterns = read_csv(f)
        patterns = patterns['Structure'].tolist()[:100] # Pour prendre seulement les 50 structures syntaxiques les plus fréquentes dans les MeSH

    terms = [t for t in phrases if t[1] in patterns]
    terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

    print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
    print("On avait {} n-grammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))


    # 2e extrant - ngrammes filtrés par patrons syntaxiques
    import pandas as pd

    def extract_terms(liste_terms, titre):
        tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
        tab.sort_values(["Fréquence"], 
                            axis=0,
                            ascending=[False], 
                            inplace=True)


        tab.to_csv(file_path + titre)

    extract_terms(terms, '_terms.csv')
    extract_terms(terms_lemmatized, '_terms-lemmatized.csv')

    # Extraction de concordances d'intérêt (KWIC - Keyword in Context)
    print("Extraction de concordances à partir de mots-clés d'intérêt (KWIC - Keyword in Context)")
    kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

    ngrammes_kwic = [" ".join([t[0].replace("' ", "'") for t in ng]) for ng in ngrammes]

    extrant = pd.DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
    kwic = {w : [] for w in kw} 

    for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
        for w in kw:
            if t.startswith(w):
                kwic[w].append(t)

    kwic = {term: FreqDist(kwic[term]) for term in kwic}

    for term in kw:
        df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
        df.sort_values(["Fréquence"], 
            axis=0,
            ascending=[False], 
            inplace=True)

        df.insert(0, 'Mot-clé', term)
    
    extrant = pd.concat([extrant, df])

    # 3e extrant - KWIC
    extrant = extrant[extrant['Fréquence'] > 30] 

    extrant.to_csv(file_path + '_KWIC.csv')

    # Extraction de termes MeSH présents dans nos données
    print("Extraction de termes MeSH présents dans nos données")
    from nltk.tokenize import MWETokenizer
    path_mesh = '../04-filtrage/mesh-fr.txt'

    with open (path_mesh, 'r', encoding='utf-8') as f:
        mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
        tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
        mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
        mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes

    extr_mesh = tokenizer_mesh.tokenize([t[0] for t in terms])

    termes_mesh = []

    for t in extr_mesh:
        if t in mesh:
            termes_mesh.append(t)

    # 4e extrant - Termes MeSH
    df = DataFrame(termes_mesh)
    df.to_csv(file_path + '_MeSH.csv')

    # Extraction de termes SNOMED présents dans nos données
    print("Extraction de termes SNOMED présents dans nos données")
    from nltk.tokenize import MWETokenizer
    path_snomed = '../04-filtrage/SNOMED_fr.csv'

    with open(path_snomed, 'r', encoding='utf-8') as f:
        sm = read_csv(f, sep=';')
        sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

        sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
        tokenizer_sm = MWETokenizer(sm, separator = ' ')

        sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

    extr_sm = tokenizer_sm.tokenize([t[0] for t in terms])

    termes_sm = []

    for t in extr_sm:
        if t in sm:
            termes_sm.append(t)

    # 5e extrant - Termes SNOMED
    df = DataFrame(termes_sm)
    df.to_csv(file_path + '_SNOMED.csv')

In [25]:
# On lance l'analyse sur le corpus global
nlp(acteur)

Corpus traité : pinel
Lire le corpus
On a un corpus de 523 documents.
Prétraitement
Tokenisation
Avec le RegExpTokenizer, notre corpus contient 794516 tokens.
Étiquetage morphosyntaxique
(Validation - On a 794506 tokens taggés).
Lemmatisation
(Validation - On a 794506 tokens lemmatisés).
Filtrage - n-grammes


In [ ]:
# Ensuite, on lance l'analyse sur les sous-corpus

path_acteur = path.join('../03-corpus/2-sous-corpus/', acteur)
liste = [tag[tag.index('_')+1:-4] for tag in os.listdir(path_acteur)]

liste

In [ ]:
for file in liste:
    nlp(file, souscorpus=True)